Pour le pipeline de pré-production, l'API est déployé sur Cloud Run et est exécuté dans un conteneur Docker. Le **pipeline de production**, qui est supposé servir des dizaines de requêtes par seconde, doit quant à lui être exécuté sur Kubernetes pour implémenter la scalabilité.

Le pipeline de production que nous allons construire ira, à terme, déployer l'API directement dans Kubernetes. Il nous faut donc construire les fichiers de configuration du Deployment qui permettra d'exécuter l'API.

<img src="https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/training/ml_engineer_facebook/img/k8s_prod6.png" />

Dans ce Notebook, il est question de mettre en place cette partie spécifique du déploiement dans le pipeline de production.

<blockquote><p>🙋 <b>Ce que nous allons faire</b></p>
<ul>
    <li>Créer et configurer un cluster K8s pour l'API de production</li>
    <li>Définir les fichiers de configuration YAML</li>
    <li>Délivrer automatiquement l'API via un déclencheur</li>
</ul>
</blockquote>

<img src="https://media.giphy.com/media/ToMjGpwbnkdSSSp0VnW/giphy.gif" />

## Configuration du cluster K8s

Pour exposer l'API via un cluster K8s, nous allons en configurer un nouveau que nous appelerons `purchase-predict-api`.

<img src="https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/training/ml_engineer_facebook/img/k8s_prod1.png" />

Nous allons ensuite configurer le pool de noeuds par défaut. Nous allons activer l'autoscaling avec un intervalle de noeuds situé entre 1 et 5. Au total, le cluster ne pourra pas excéder de 5 noeuds.

<img src="https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/training/ml_engineer_facebook/img/k8s_prod2.png" />

Concernant les types de machines, nous pouvons garder les `e2-medium`.

<img src="https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/training/ml_engineer_facebook/img/k8s_prod3.png" />

Puisque nous sommes limité à 5 noeuds en autoscaling, nous pourrons avoir au total 10 CPU et 10 Go. Nous affectons à chaque noeud 10Go d'espace disque, ce qui sera amplement suffisant puisqu'il n'y a pas de stockage local.

Une fois le cluster lancé, nous pouvons nous y connecter via la commande `gcloud`.

<img src="https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/training/ml_engineer_facebook/img/k8s_prod4.png" />

### Espaces de noms

En pratique, il se peut que les clusters K8s ne contiennent pas un seul mais plusieurs applications. Dans ce genre de situation, une bonne pratique consiste à segmenter les ressources présentes à l'aide d'espaces de noms (*namespaces*). Ces espaces de noms fournissent des isolations logiques entre les ressources. Les services d'un espace de nom ne pourra être associé à des pods d'un autre service.

Par défaut, il y a 4 espaces de noms présents dans le cluster.

L'espace de noms `default` est celui utilisé par défaut lorsque l'on ne spécifie pas d'espaces de noms pour les ressources. Les autres espaces de noms `kube-*` sont utilisés par le cluster lui-même.

<div class="alert alert-block alert-info">
    Il faut éviter d'utiliser des espaces de noms avec <code>kube</code> comme préfixe, car ils sont réservés au cluster.
</div>

Créons le fichier `ns.yaml` qui va nous permettre de définir un espace de noms.

Comme toujours, pour créer la ressource, nous utilisons `kubectl apply`.

L'espace de noms étant créé, nous pouvons le détailler.

Les quotas appliqués sont ceux configurés par défaut dans GKE. Il est bien entendu possible de modifier ces valeurs dans la configuration du cluster, mais elles seront amplement suffisantes pour l'API.

La configuration du cluster K8s est terminée, nous pouvons maintenant nous atteler aux fichiers de configurations.

## Fichiers de configuration YAML

Avant de commencer, assurons-nous d'être sur **la bonne branche** Git ! Le fichier `cloudbuild.yaml` qui nous avions configuré était valide pour la pré-production, mais ne sera plus identique pour la production.

Pour cela, nous allons retourner sur la branche `master` et **fusionner** avec la branche `staging`.

Créons un dossier `k8s/` à la racine du projet : nous allons y insérer tous les fichiers de configuration YAML qui seront utilisés avec `kubectl`. Commençons par le fichier `deployment.yaml`.

La configuration du Deployment est très similaire à celui que nous avions déjà réalisé, à la seule différence que nous avons modifié l'image Docker, les ressources et des variables d'environnement nécessaires pour l'exécution de l'API. L'image Docker correspond ici au tag le plus récent.

<img src="https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/training/ml_engineer_facebook/img/k8s_prod5.png" />

Ajoutons le Deployment dans le cluster.

En faisant un `kubectl get pods`, nous voyons qu'il n'y a aucune ressource ! En effet, sans mention particulière, `kubectl` ira toujours chercher les ressources de l'espace de noms `default`. Il faut donc le spécifier pour accéder aux ressources.

<div class="alert alert-block alert-info">
    Il se peut que le cluster procède à de l'autoscaling : le deuxième pod prendrai plus de temps à démarrer.
</div>

Maintenant que nos pods sont en place, créons le service associé dans le fichier `service.yaml`.

Pour terminer, nous allons placer un Ingress en frontal.

Comme nous l'avions fait précédemment, l'Ingress est en cours de création comme nous pouvons le voir avec `kubectl get ingress app-ingress --namespace api-purchase`.

Il faut patienter quelques minutes le temps que Google provisionne un équilibreur de charge et qu'il soit entièrement configuré. Ensuite, nous pouvons requêter à l'adresse renseignée par l'Ingress via `kubectl get ingress --namespace api-purchase`.

Maintenant que toutes nos ressources sont en place, exécutons le code Python suivant.

In [ ]:
import os
import requests
import pandas as pd

dataset = pd.read_csv(os.path.expanduser("~/data/primary.csv"))
dataset = dataset.drop(["user_session", "user_id", "purchased"], axis=1)

In [ ]:
requests.post(
    "http://34.117.58.199/predict",  # Remplacer par l'adresse IP associée à l'Ingress
    json=dataset.sample(n=10).to_json()
).json()

Notre API est correctement déployée sur Kubernetes ! 👏

Il ne reste plus qu'à automatiser le déploiement de l'API via un déclencheur Cloud Build.

## Configuration de Cloud Build

Puisque nous sommes sur la branche `master`, il ne faut plus déployer l'image Docker vers Cloud Run. La dernière étape du fichier `cloudbuild.yaml` doit être supprimée.

Comme pour la pré-production, le déclencheur va construire l'image Docker et l'envoyer vers le registre de conteneurs du projet Google Cloud. Il nous faut rajouter les autres étapes qui vont prendre cette image Docker pour l'insérer dans les configurations Kubernetes. Pour rappel, l'image Docker se voit attribuer un tag qui correspond au 7 premières lettres du commit SHA de Git.

> ❓ Comment utiliser ce tag dans le fichier de configuration YAML ?

Et oui, dans notre fichier `deployment.yaml`, nous devons spécifier le tag Docker à utiliser. Comment pourrions-nous faire référence à un tag *dynamique* déterminé lors du déclenchement du build ?

La solution consiste à utiliser du **templating**.

### Templating de fichier

Le templating est un procédé qui consiste à écrire des références à des macros dans des fichiers de configuration, puis à remplacer ces macros par des valeurs dynamiques lors du rendu. L'objectif est de pouvoir écrire des fichiers de configuration qui, à tout moment, peuvent être dynamiques en remplaçant certaines valeurs selon des variables écrites dans le texte. Ce procédé de templating est également présent dans d'autres applications, dont Apache Airflow que nous verrons très bientôt.

L'idée ici, c'est de ne pas utiliser un tag spécifique dans le fichier de Deployment, mais de faire référence à une macro qui sera ensuite remplacée au moment de l'exécution. Dans le fichier Deployment, nous modifions le tag en remplaçant par la macro `DOCKER_TAG`.

Nous pouvons ensuite construire la prochaine étape du fichier `cloudbuild.yaml`.

Analysons en détails cette étape. Elle fait appel à des commandes Bash.

- La première commande `sed` va avoir pour objectif de concaténer l'ensemble des fichiers d'extension `*.yaml` à l'intérieur du dossier `k8s/` et de séparer ces concaténations par des sauts de lignes suivies de trois tirets. Le résultat est ensuite enregisté dans le fichier `config.yaml.tpl`.
- Encore avec la commande `sed`, on remplace la macro `DOCKER_TAG` du fichier `config.yaml.tpl` par les 7 premières lettres du commit SHA de Git, faisant référence au tag de l'image Docker construite par les étapes précédentes. Le rendu du templating est ensuite enregistré dans le fichier `config.yaml`.

L'intérêt d'avoir un seul fichier `config.yaml` c'est qu'il va à la fois contenir le tag de l'image Docker construite juste avant, mais aussi car la mise à jour des ressources sur K8s sera plus facile si toutes les configurations sont centralisées dans un seul fichier.

La dernière étape concerne le déploiement sur K8s.

L'environnement de build est `kubectl`, qui contient déjà toutes les dépendances pour l'interface K8s. Nous utilisons un `apply` pour mettre à jour les ressources, en précisant l'espace de noms.

Les variables d'environnements `CLOUDSDK_COMPUTE_ZONE` et `CLOUDSDK_CONTAINER_CLUSTER` sont nécessaires pour que Cloud Build sache où et vers quel cluster exécuter la mise à jour des ressources.

Au final, le fichier `cloudbuild.yaml` contient 4 étapes.

Nous pouvons maintenant créer un nouveau déclencheur sur <a href="https://console.cloud.google.com/cloud-build/triggers" target="_blank">Cloud Build</a>.

<img src="https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/training/ml_engineer_facebook/img/k8s_prod7.png" />

Avant de lancer une exécution, il est important d'autoriser Cloud Build à interagir avec GKE, comme nous l'avions fait pour Cloud Run.

<img src="https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/training/ml_engineer_facebook/img/k8s_prod9.png" />

Ajoutons les fichiers au référentiel Git puis poussons les nouvelles références.

Après quelques minutes d'exécution, le build devrait correctement se finaliser.

<img src="https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/training/ml_engineer_facebook/img/k8s_prod8.png" />

En affichant les pods avec `kubectl get pods --namespace api-purchase`, nous voyons qu'ils sont beaucoup plus récents.

Pour s'assurer qu'il s'agit de la bonne image Docker en cours d'exécution, nous pouvons chercher l'information en affichant le YAML du Deployment.

Sur <a href="https://console.cloud.google.com/gcr/images">Container Registry</a>, il s'agit bien de l'image la plus récente.

<img src="https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/training/ml_engineer_facebook/img/k8s_prod10.png" />

Notre déploiement vers le cluster K8s est maintenant pleinement opérationnel.

## ✔️ Conclusion

Nous y sommes presque ! Plus qu'une dernière étape pour avoir notre pipeline de production.

- Nous avons configuré un cluster K8s pour l'environnement de production.
- Nous avons défini des fichiers de configuration YAML.
- Le déploiement de l'API a été automatisé via Cloud Build.

> ➡️ Il ne reste plus qu'à construire le pipeline de production en entier pour avoir un environnement 100% automatisé.